MOUNT GOOGLE DRIVE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


IMPORT LIBRARIES

In [2]:
import pandas as pd

In [3]:
import os

In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

GENERATING FEATURES FROM IMAGES AND STORING IN CSV

In [ ]:
char_arr = []
area_arr=[]
perimeter_arr=[]
RND_arr=[]
center_x=[]
center_y=[]
ratio_arr=[]
for j in range(26):
  #directory path for every alphabet
  folder_dir="/content/drive/MyDrive/test_set/"+str(chr(65+j))
  #reading all images in a loop
  for img in os.listdir(folder_dir):
    f = os.path.join(folder_dir, img)
    #reading images from cv2 and storing in img
    img =cv2.imread(f)
    #converting into grey scale
    image =cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #finding contour in images
    contours,hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #if detect only then this
    if(len(contours)==1):
       contoue = np.squeeze(contours[0]).copy()
       contoue1=np.copy(contours)
    #if detects more then 1 take the contour having the max length
    else:
      ma1x=0
      for i in range(len(contours)):
         if(len(contours[i])>ma1x):
           contoue=np.squeeze(contours[i]).copy()
           ma1x=len(contours[i])
           contoue2=np.copy(contours[i])
           contoue1=[]
           contoue1.append(contoue2)
    for i in range(contours[0].shape[0]):
        contoue[i][1]=contoue[i][1]-32
    for i in range(contours[0].shape[0]):
         if(contoue[i][1]<0):
             contoue[i][1]=abs(contoue[i][1])
             contoue[i][1] = contoue[i][1] + 32
         else:
            contoue[i][1]=-contoue[i][1]
            contoue[i][1] = contoue[i][1] + 32
    for cnt in contours:
        epsilon = 0.01*cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        img = cv2.drawContours(img, [approx], -1, (0,225,0), 1)
    for cnt in contoue1:
        #calculating center,area,perimeter
        M = cv2.moments(cnt)
        cx = float(M['m10']/M['m00'])
        cy = float(M['m01'] / M['m00'])
        center_x.append(cx)
        center_y.append(cy)
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt,True)
    y=contoue.copy()
    #calculating ratio
    x=y[:,0]
    y=y[:,1]
    w=max(x)-min(x)
    h=max(y)-min(y)
    ratio=h/w
    #calculating roundness
    RND=(4*3.14*area)/(perimeter*perimeter)
    #storing the results
    char_arr.append(str(chr(65+j)))
    area_arr.append(area)
    perimeter_arr.append(perimeter)
    RND_arr.append(RND)
    ratio_arr.append(ratio)

MAKING DICTIONARY OF RESULTS

In [ ]:
dict = {'char':char_arr,'area':area_arr,'perimeter':perimeter_arr,'roundness':RND_arr, 'ratio':ratio_arr, 'center_x':center_x, 'center_y':center_y}

MAKING DATAFRAME OF DICTIONARY

In [ ]:
df = pd.DataFrame(dict)

In [ ]:
print(df)

     char    area   perimeter  roundness     ratio   center_x   center_y
0       A  1454.0  165.396969   0.667573  1.225000  29.698533  38.675378
1       A  1506.5  165.982755   0.686804  1.090909  28.652395  41.077221
2       A  1324.0  163.053823   0.625483  1.022727  30.107628  39.128399
3       A  1680.0  173.396969   0.701804  1.065217  28.899802  38.076488
4       A  1400.5  163.639609   0.656895  1.256410  29.642985  38.826253
...   ...     ...         ...        ...       ...        ...        ...
6495    Z  1024.5  137.497474   0.680632  1.228571  32.561249  45.669432
6496    Z  1171.5  163.639609   0.549484  1.416667  31.568502  37.626974
6497    Z  1043.5  151.639609   0.569976  1.184211  30.239898  43.323910
6498    Z  1179.5  159.982755   0.578817  1.457143  31.380246  37.542038
6499    Z  1024.0  140.911687   0.647732  1.200000  32.429688  45.902832

[6500 rows x 7 columns]


MAKING CSV FILE

In [ ]:
df.to_csv('train.csv')